In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(42)

Example: How it works

In [19]:
lstm = nn.LSTM(3,3)
inputs = [torch.randn(1,3) for _ in range(5)]
hidden = (torch.randn(1,1,3),
         torch.randn(1,1,3))
for i in inputs:
    out, hidden = lstm(i.view(1,1,-1), hidden)
print(out)
print(hidden)

tensor([[[-0.0798, -0.2567, -0.0705]]], grad_fn=<StackBackward>)
(tensor([[[-0.0798, -0.2567, -0.0705]]], grad_fn=<StackBackward>), tensor([[[-0.1253, -0.5394, -0.1118]]], grad_fn=<StackBackward>))


In [20]:
inputs = torch.cat(inputs).view(len(inputs),1,-1)
hidden = (torch.randn(1,1,3), torch.randn(1,1,3))
out, hidden = lstm(inputs, hidden)
print(out)
print(hidden)

tensor([[[-0.2625, -0.0035,  0.0144]],

        [[-0.1817,  0.0256,  0.0357]],

        [[-0.0763, -0.0414, -0.0182]],

        [[-0.0459, -0.0906, -0.0182]],

        [[-0.1696, -0.1274, -0.0801]]], grad_fn=<StackBackward>)
(tensor([[[-0.1696, -0.1274, -0.0801]]], grad_fn=<StackBackward>), tensor([[[-0.2716, -0.2572, -0.1213]]], grad_fn=<StackBackward>))


Example: An LSTM for part-of-speech tagging

In [21]:
def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)

In [24]:
training_data = [
    ('The dog ate the apple'.split(), ['DET', 'NN', 'V', 'DET', 'NN']),
    ('Everybody read that book'.split(), ['NN', 'V', 'DET', 'NN'])
]
word_to_ix = {}
for sent, tags in training_data:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
print(word_to_ix)
tag_to_ix = {'DET': 0, 'NN': 1, 'V': 2}
EMBEDDING_DIM = 6
HIDDEN_DIM = 6

{'The': 0, 'dog': 1, 'ate': 2, 'the': 3, 'apple': 4, 'Everybody': 5, 'read': 6, 'that': 7, 'book': 8}


In [28]:
class LSTMTagger(nn.Module):
    
    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim
        
        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)
        
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)
        
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)
        self.hidden = self.init_hidden()
        
    def init_hidden(self):
        return (torch.zeros(1,1,self.hidden_dim),
                torch.zeros(1,1,self.hidden_dim))
    
    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, self.hidden = self.lstm(embeds.view(len(sentence),1,-1), self.hidden)
        tag_space = self.hidden2tag(lstm_out.view(len(sentence),-1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

In [29]:
model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(tag_to_ix))
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

In [30]:
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)
    print(tag_scores)

tensor([[-0.8317, -1.3154, -1.2163],
        [-0.8162, -1.3694, -1.1920],
        [-0.8941, -1.2314, -1.2069],
        [-0.8988, -1.2664, -1.1677],
        [-0.8816, -1.3416, -1.1257]])


In [37]:
for epoch in range(300):
    for sentence, tags in training_data:
        model.zero_grad()
        
        model.hidden = model.init_hidden()
        
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = prepare_sequence(tags, tag_to_ix)
        
        tag_scores = model(sentence_in)
        
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()

In [41]:
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)
    print(tag_scores)

tensor([[-7.1916e-02, -3.0935e+00, -3.7277e+00],
        [-5.2014e+00, -5.9648e-03, -7.7339e+00],
        [-3.0953e+00, -6.5773e+00, -4.7775e-02],
        [-3.1873e-02, -4.6398e+00, -3.8299e+00],
        [-4.4296e+00, -1.4331e-02, -6.0709e+00]])


Predicted tags are 0, 1, 2, 0, 1. These are the correct tags!